In [1]:
from SpinTrain import *


Sz = [Operator('S%d'%i, 'S^z_{%d}'%i) for i in [6, 1,2,3,4,5]]

a,b,c,d = [Operator('%s'%i, 'S^z_{%s}'%i) for i in 'abcd']



a_up = Operator('a⁺', 'S^+_{a}')
a_dn = Operator('a⁻', 'S^-_{a}')

b_up = Operator('b⁺', 'S^+_{b}')
b_dn = Operator('b⁻', 'S^-_{b}')

ca = CommutatorAlgebra()

def define_spinalg(plus, minus, z):
    ca.set_commutator(plus, minus)(2*z)
    ca.set_commutator(z, plus)( plus )
    ca.set_commutator(z, minus)( -1*minus )

In [2]:
define_spinalg(a_up, a_dn, a)
define_spinalg(b_up, b_dn, b)

for o in [a,b,c,d] + Sz:
    ca.add_operator(o)
# effectively scalars, idk why I bother

In [3]:
# define the Z objects

bond_zetas = [Operator('ζ%d'%i,r'\zeta_%d'%i,scalar=True) for i in range(1,4)]
bond_zetas_bar = [Operator('ζb%d'%i,r'\zeta_%d^*'%i,scalar=True) for i in range(1,4)]

def zeta(i):
    return bond_zetas[(i-1)%3]
def zeta_bar(i):
    return bond_zetas_bar[(i-1)%3]

KA = Operator('KA','K_A',scalar=True)
KB = Operator('KB','K_B',scalar=True)

def conjugate_zetas(x):
    repl = []
    for i in range(3):
        repl.append((bond_zetas[i], bond_zetas_bar[i]))
        repl.append((bond_zetas_bar[i], bond_zetas[i]))
    return x.replaceall(*repl)


def order_scalars(x):
    desired_order = [KA, KB] + bond_zetas + bond_zetas_bar + Sz + [a,b,c,d]
    for o in reversed(desired_order):
        ca.move_left(x, o)

Za = KB*(zeta(1)*Sz[1] + zeta(2)*Sz[2] + zeta(3)*Sz[3]) + KA*(zeta(1)*b + zeta(2)* c + zeta(3)*d)
Zb = KB*(zeta(1)*Sz[4] + zeta(2)*Sz[5] + zeta(3)*Sz[0]) + KA*(zeta(1)*a + zeta(2)* d + zeta(3)*c)

Za_bar = conjugate_zetas(Za)
Zb_bar = conjugate_zetas(Zb)


hh = a_up*b_dn
kp = a_dn*Za_bar
km = b_up*Zb

hh_dag = a_dn*b_up
kp_dag = a_up*Za
km_dag = b_dn*Zb_bar

In [4]:
S_total =  hh*kp*km + kp*km*hh + km*hh*kp + hh*km*kp + km*kp*hh + kp*hh*km
S_total += hh_dag*kp_dag*km_dag + kp_dag*km_dag*hh_dag + km_dag*hh_dag*kp_dag + hh_dag*km_dag*kp_dag + km_dag*kp_dag*hh_dag + kp_dag*hh_dag*km_dag

order_scalars(S_total)

ca.move_right(S_total,a_up)
ca.move_right(S_total,a_dn)

ca.move_right(S_total,b_up)
ca.move_right(S_total,b_dn)


x1 = S_total.replaceall((a_up*a_dn, a+Fraction(1,2)),(b_up*b_dn, b+Fraction(1,2)))
x1.collect()


In [5]:
w = Operator('w','w',scalar=True)
w_bar = Operator('wb',r'w^*',scalar=True)

def simplify_zetas(expr):
    # only works if scalars are in canonical order
    simpl = []
    for i, z in enumerate(bond_zetas):
        for j, zb in enumerate(bond_zetas_bar):
            if i == j:
                simpl.append((z*zb, 1))
            elif i == (j+1) %3:
                simpl.append((z*zb, w))
            else:
                simpl.append((z*zb, w_bar))
    return expr.replaceall(*simpl).replaceall((w*w_bar, 1), (w_bar*w, 1))


x2 = simplify_zetas(x1)
order_scalars(x2)
x2.collect()
x2.show()

<IPython.core.display.Latex object>

In [6]:
Zops = [a,b,c,d] + Sz


x3 = x2.replaceall( *[(z*z,Fraction(1,4)) for z in Zops])
x3.collect()
x3.show()

<IPython.core.display.Latex object>

In [7]:
root3 = Operator('√3',r'\sqrt{3}',scalar=True)

def convert_to_rectangular(expr):
    ca.move_right(expr, w)
    ca.move_right(expr, w_bar)
    coeff_w=expr.coefficient(w)
    coeff_wb=expr.coefficient(w_bar)
    
    remainder = expr + -1*(coeff_w*w +coeff_wb*w_bar)
    remainder.collect()
    real = remainder + Fraction(-1,2)*(coeff_w + coeff_wb)
    imag = Fraction(1,2)*(coeff_w + -1*coeff_wb)*root3
    return real, imag


x4, im = convert_to_rectangular(x3)
assert im == 0

In [8]:
x4.collect()
x4.show()

<IPython.core.display.Latex object>

In [9]:
# enforce an ice rule

x5 = x4.replaceall((a*b*c*d, Fraction(1,16)),(a*b*c, Fraction(1,4)*d), (a*b*d, Fraction(1,4)*c),(a*c*d, Fraction(1,4)*b),(b*c*d, Fraction(1,4)*a))
x5.collect()
x5.sort('multiplier')
x5.show()

<IPython.core.display.Latex object>

In [10]:
from itertools import permutations

quartic = Expression()
quad = Expression()

for t in x5.terms:
    if len(t) == 6:
        quartic += t
    else:
        quad += t
    
assert quad + quartic == x5

In [11]:
quad.show()

<IPython.core.display.Latex object>

In [12]:
quartic.show()

<IPython.core.display.Latex object>

In [13]:
coeff_aa = quad.coefficient(KA*KA, 'right')
coeff_aa.show()

<IPython.core.display.Latex object>

In [14]:
coeff_ab = quad.coefficient(KA*KB, 'right')
dd = quad +(KA*KA*coeff_aa+KA*KB*coeff_ab)*-1
dd.collect()
print(dd)
coeff_ab.show()

  -3/2 KB KB S1 S5  -3/2 KB KB S6 S2  -3/2 KB KB S3 S4  -3/2 KB KB S6 S1  -3/2 KB KB S2 S4  -3/2 KB KB S3 S5  +3 KB KB S1 S4  +3 KB KB S2 S5  +3 KB KB S6 S3


<IPython.core.display.Latex object>